In [1]:
import pandas as pd
import os
from clean_data_for_decision_tree import *

In [2]:
dir = '../../data/labeled_csv_files'
file = 'Anatel_labeled.csv'
file_path = os.path.join(dir, file)
anatel = pd.read_csv(file_path)

In [3]:
get_rid_of_problematic_columns(anatel)
rename_anatel_cols(anatel)


# Handle Missing Data

In [4]:
perc_null = (anatel.isnull().mean() * 100).round(2)


### Alternative 1 - Getting rid of all missing values

In [5]:
anatel_no_na = anatel.dropna()

In [6]:
print(f"Length of original dataframe: {len(anatel)}")
print(f"Lenght of non-nan dataframe: {len(anatel_no_na)}")

anatel = anatel_no_na.copy()
del anatel_no_na

Length of original dataframe: 57328
Lenght of non-nan dataframe: 55006


# Scaling/Normalizing

Scaling and normalizing are not strictly necessary for decision trees. Here's why:

**Decision trees are insensitive to monotonic transformations:**

* Decision trees make splitting decisions based on the relative order of features, not their absolute values.
* Scaling (e.g., MinMaxScaler) only changes the range of values, not their order.
* Therefore, scaling doesn't affect how the tree splits the data.

**However, there are some situations where scaling or normalizing might be beneficial:**

* **Improves numerical stability:** Scaling features to a similar range can improve the numerical stability of the algorithm.
* **Enhances visualization and interpretation:** Scaled features are easier to compare and interpret, especially when dealing with mixed units.
* **Helps with comparing with other algorithms:** If you plan to combine decision trees with other algorithms that require scaled data, scaling beforehand can simplify the process.
* **May improve performance in some cases:** Although not guaranteed, scaling can sometimes lead to slightly better performance for decision trees.

**Normalization, however, requires caution:**

* Normalization (e.g., StandardScaler) changes the distribution of features, which can affect the splitting decisions.
* This may lead to suboptimal tree structures and potentially worse performance.

**In summary:**

* Scaling is generally not necessary for decision trees but can be beneficial in some situations.
* Normalization should be used cautiously and with awareness of its potential impact.
* It's recommended to experiment and compare performance with and without scaling/normalization in your specific case.


# Encoding Categorical Variables

If the dataset includes categorical variables, you'll need to encode them because decision trees typically work with numeric data.

In [7]:
for col in anatel.columns:
    print(f"Column '{col}': {anatel[col].dtype}")

Column 'Station': float64
Column 'MinTxFreq': float64
Column 'MaxTxFreq': float64
Column 'MinRxFreq': float64
Column 'MaxRxFreq': float64
Column 'SiteType': object
Column 'AntennaCode': int64
Column 'AntennaGain': float64
Column 'FrontBackAntennaRation': float64
Column 'AnguloMeiaPotenciaAntena_max': float64
Column 'ElevationAngle': float64
Column 'Polarization': object
Column 'AntennaHeight': float64
Column 'TransmitterPower': float64
Column 'NecessaryBandwidth': float64
Column 'BasicFeatures': object
Column 'LTE': bool
Column 'WCDMA': bool
Column 'GSM': bool
Column 'NR_NSA': bool
Column 'NR_SA-NSA': bool
Column 'DMR': bool
Column 'Digital': bool
Column 'DaysSinceLicensing': float64
Column 'DaysSinceFirstLicensing': float64
Column 'DaysUntilExpiration': int64


- [x] SiglaUf_max
- [x] CodMunicipio_max
- [x] CodTipoClasseEstacao_max
- [x] ClassInfraFisica_agg_non_none
- [x] Polarizacao_max
- [x] CodDebitoTFI_max
- [x] CaracteristicasBasicas_agg_non_none
- [x] LTE_max
- [x] WCDMA_max
- [x] GSM_max
- [x] NR_NSA_max
- [x] NR_SA-NSA_max
- [x] DMR_max
- [x] Digital_max

In [8]:
anatel = pd.get_dummies(anatel, columns=['Polarization'], prefix='Polarization')
anatel = pd.get_dummies(anatel, columns=['BasicFeatures'], prefix='BasicFeatures')


Decision trees and random forests can handle boolean variables without encoding. They naturally make binary decisions based on the values of the features.

In [9]:
del dir, col, perc_null, file_path, file

# Build and Train the Decision Tree Model

In [10]:
anatel['Station'] = anatel['Station'].astype(int)
anatel.set_index('Station', inplace=True)

In [11]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

### Split Data into Features and Target

In [13]:
X = anatel.drop("SiteType", axis=1)
y = anatel["SiteType"]

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
tree_model = DecisionTreeClassifier(random_state=42)
tree_model.fit(X_train, y_train)

DecisionTreeClassifier(random_state=42)

In [17]:
y_pred = tree_model.predict(X_test)

In [ ]:
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

In [ ]:
from sklearn.tree import plot_tree
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 8))
plot_tree(tree_model, filled=True, feature_names=X_encoded.columns, class_names=tree_model.classes_)
plt.show()

tuned_tree_model = DecisionTreeClassifier(max_depth=5, min_samples_split=2, min_samples_leaf=1, random_state=42)
tuned_tree_model.fit(X_train, y_train)

